In [1]:
pip install transformers torch torchvision datasets pillow gradio streamlit accelerate

In [2]:
import os
import random
import torch
import pandas as pd
import gradio as gr
import streamlit as st
from PIL import Image, ImageDraw
from transformers import CLIPProcessor, CLIPModel, TrainingArguments, Trainer
from torch.nn.functional import cross_entropy
from datasets import Dataset

In [3]:
image_dir = 'synthetic_images'
os.makedirs(image_dir, exist_ok=True)

In [4]:
for i in range(10):
    img = Image.new('RGB', (224, 224), color=(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
    d = ImageDraw.Draw(img)
    d.text((10, 10), f'Sample Image {i}', fill=(255, 255, 255))
    img.save(os.path.join(image_dir, f'image_{i}.jpg'))


In [5]:
captions = [f'This is a caption for image {i}' for i in range(10)]


In [6]:
data = {'image_path': [os.path.join(image_dir, f'image_{i}.jpg') for i in range(10)], 'caption': captions}
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

In [7]:
df

,image_path,caption
0,synthetic_images/image_0.jpg,This is a caption for image 0
1,synthetic_images/image_1.jpg,This is a caption for image 1
2,synthetic_images/image_2.jpg,This is a caption for image 2
3,synthetic_images/image_3.jpg,This is a caption for image 3
4,synthetic_images/image_4.jpg,This is a caption for image 4
5,synthetic_images/image_5.jpg,This is a caption for image 5
6,synthetic_images/image_6.jpg,This is a caption for image 6
7,synthetic_images/image_7.jpg,This is a caption for image 7
8,synthetic_images/image_8.jpg,This is a caption for image 8
9,synthetic_images/image_9.jpg,This is a caption for image 9


In [8]:
dataset

Dataset({
    features: ['image_path', 'caption'],
    num_rows: 10
})

In [9]:
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [10]:
from transformers import CLIPProcessor, CLIPModel

# Load the CLIP processor and model
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def preprocess(examples):
    images = [Image.open(image_path) for image_path in examples['image_path']]
    texts = examples['caption']
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation=True)
    return inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [12]:
import torch
from transformers import TrainingArguments, Trainer
from torch.nn.functional import cross_entropy

In [13]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(len(logits_per_image), device=logits_per_image.device)

        loss_img = cross_entropy(logits_per_image, labels)
        loss_txt = cross_entropy(logits_per_text, labels)
        loss = (loss_img + loss_txt) / 2

        return (loss, outputs) if return_outputs else loss

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",  # Evaluate every epoch
    num_train_epochs=3,
    save_steps=10,
    save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
def fine_tune_model():
    # Initialize the custom trainer
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=processor,
    )

    # Fine-tune the model
    trainer.train()

# Fine-tune the model
fine_tune_model()

Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


In [16]:
def predict(image, caption):
    inputs = processor(text=[caption], images=[image], return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    logits_per_text = outputs.logits_per_text
    return {
        'logits_per_image': logits_per_image.tolist(),
        'logits_per_text': logits_per_text.tolist(),
    }

In [17]:
st.title("CLIP Model Fine-tuning and Prediction")

uploaded_image = st.file_uploader("Upload an image...", type=["jpg", "jpeg", "png"])
caption = st.text_input("Enter a caption for the image:")

2024-06-25 13:15:36.051 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-25 13:15:36.054 Session state does not function when running a script without `streamlit run`


In [18]:
if uploaded_image is not None and caption:
    image = Image.open(uploaded_image)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Make prediction
    if st.button("Predict"):
        prediction = predict(image, caption)
        st.write("Logits per Image:", prediction['logits_per_image'])
        st.write("Logits per Text:", prediction['logits_per_text'])

# Sidebar with training information
st.sidebar.title("Training Information")
st.sidebar.write("Training Loss:")
st.sidebar.write("Validation Loss:")